### Descripción de la base de datos

La base de datos seleccionada para este proyecto fue la base [The Extended Yale Face Dataset B](http://vision.ucsd.edu/~iskwak/ExtYaleDatabase/ExtYaleB.html) , en específico las versión recortada utilizada en el artículo [Acquiring Linear Subspaces for Face Recognition under Variable Lighting](http://vision.ucsd.edu/~leekc/papers/9pltsIEEE.pdf). Estas versiones recortadas resultan mejores para este tipo de algoritmos pues dentro de la foto sólo aparece la región entre las orejas y de la frente a la barbilla, así evitamos generar ruido innecesario producido por el cabello, ropa o joyería utilizada por la persona a la que se le toma la foto.
 
La manera en que se recolectaron las fotos de este dataset fue con la ayuda de un domo geodésico que tenía varias lámparas alrededor programadas para activarse en distintos momentos en un intervalo de 2 segundos para generar 64 distintas configuraciones de iluminación. En el centro de este domo se sentaba a la persona que se iba a fotografiar y al activarse la secuencia de iluminación se tomaron 64 fotografías de cada persona. Este procedimiento se repitió para 38 personas.

Cada fotografía en su versión recortada tiene dimensiones de 168 x 192 pixeles lo que, una vez vectorizada la imagen, resulta en un vector con 32,256 entradas. Multiplicando esto por las 64 fotos de las 38 personas, obtenemos una matriz con aproximadamente 78 millones de entradas.

### Algoritmo de Eigenfaces

El problema de las eigenfaces es un problema relativamente viejo, tuvo sus inicios en 1987 con Sirovich y Kirby en su artículo [Low-dimensional procedure for the characterization of human faces](http://pdfs.semanticscholar.org/2a62/d0cca2fabf1d6f6ee15e4c14cef415b657d1.pdf) pero no fue si no hasta 1991 que se estudió su aplicación en reconocimiento facial automatizado con Turk y Pentland en su artículo [Eigenfaces for Recognition](https://www.mitpressjournals.org/doi/pdf/10.1162/jocn.1991.3.1.71).

En este caso, nuestro conjunto de datos tiene 38 personas, tomamos 36 de estas para nuestro conjunto de entrenamiento y resguardamos las 2 restantes para el conjunto de prueba.

Antes de comenzar a trabajar, y como se mencionó en la sección anterior, es necesario pasar cada una de las imágenes a un vector columna y colocar cada uno de estos en una matriz. Posteriormente, obtenemos el vector columna promedio de esta matriz y se lo restamos a todos los vectores para así centrar nuestros datos.

![title](../images/matriz-x.png)

Así, el calcular la SVD de la matriz de fotos centrada se es realmente un problema de cálculo de PCA. Una vez calculada la SVD y tomando en cuenta la notación usual, la matriz U representa un **ESPACIO?** donde cada uno de sus vectores columna son vectores que conforman una base ortonormal de este espacio , también conocidos como eigenfaces.

![title](../images/matriz-u.png)